In [1]:
import os
import numpy as np
import tensorflow as tf
import cv2
import timeit

In [2]:
class Predictor:
    """
    This class loads and runs the tensorflow model
    """
    def __init__(self, frozen_graph, gpu):
        """
        frozen_graph: the path to the model
        gpu: which gpu to use
        """
        # load model into detection_graph
        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
        detection_graph = tf.Graph()
        with detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(frozen_graph, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
        self.graph = detection_graph
        
        with self.graph.as_default():
            config = tf.ConfigProto()
            config.gpu_options.allow_growth=True
            self.sess = tf.Session(config=config)
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            self.tensor_dict = {}
            for key in ['detection_scores','detection_classes', 'detection_boxes']:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    self.tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
            self.image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
        
    def timeit(self, images, number):
        """
        images: one image resized to the correct size (e.g. 640x640)
        number: how often the speedmeasurement should be done
        """
        with self.graph.as_default():
            # warmup
            self.sess.run(self.tensor_dict, feed_dict={self.image_tensor: np.random.randint(0, 255, images.shape)})
            return timeit.timeit(lambda: self.sess.run(self.tensor_dict, feed_dict={self.image_tensor: images}), number=number)/number*1000

In [3]:
img = cv2.imread('livedemo/d.jpg')
img = cv2.resize(img[:, :img.shape[0], ::-1], (640, 640))
imgs = cv2.resize(img, (300, 300))

In [4]:
#SSD
p = Predictor('train/ssd_mobilenet_alex/1/frozen_inference_graph.pb', 0)
print('%.2fms' % p.timeit(imgs[np.newaxis], 1))
print('%.2fms' % p.timeit(imgs[np.newaxis], 10))
del p

16.24ms
12.44ms


In [5]:
#SSD+FPN
p = Predictor('train/ssd_julius_mobilefpn_large/frozen_inference_graph.pb', 0)
print('%.2fms' % p.timeit(img[np.newaxis], 1))
print('%.2fms' % p.timeit(img[np.newaxis], 10))
del p

31.32ms
29.23ms


In [6]:
#RFCN
p = Predictor('train/rfcn/9/frozen_inference_graph.pb', 0)
print('%.2fms' % p.timeit(img[np.newaxis], 1))
print('%.2fms' % p.timeit(img[np.newaxis], 10))
del p

59.69ms
62.79ms


In [7]:
# And now with the test images
data = np.load('val_data.npz')
img = data['imgs']
img = np.concatenate([cv2.resize(img[i], (640, 640))[np.newaxis] for i in range(len(img))])
imgs = np.concatenate([cv2.resize(img[i], (300, 300))[np.newaxis] for i in range(len(img))])

In [8]:
#SSD
p = Predictor('train/ssd_mobilenet_alex/1/frozen_inference_graph.pb', 0)
print('%.2fms' % (p.timeit(imgs, 1) / len(imgs)))
del p

5.89ms


In [9]:
#SSD+FPN
#batchsize of 85 seems to be to large
p = Predictor('train/ssd_julius_mobilefpn_large/frozen_inference_graph.pb', 0)
print('%.2fms' % (p.timeit(img, 1) / len(img)))
del p

19.63ms


In [10]:
#RFCN
p = Predictor('train/rfcn/9/frozen_inference_graph.pb', 0)
print('%.2fms' % (p.timeit(img, 1) / len(img)))
del p

36.94ms
